In [1]:
import wooldridge
df = wooldridge.data('jtrain2')
#df['avg'] = 0.5 * (df.re74+df.re75)
df = df.dropna()
#df = df[df.avg <= 15]
y = df.re78
d = df.train
x = df[['age', 'educ', 'black', 'hisp', 'married', 're74', 're75']]
print(df.shape)
df.head()

(445, 19)


,train,age,educ,black,hisp,married,nodegree,mosinex,re74,re75,re78,unem74,unem75,unem78,lre74,lre75,lre78,agesq,mostrn
0,1,37,11,1,0,1,1,13,0.0,0.0,9.93005,1,1,0,0.0,0.0,2.295566,1369,13
1,1,22,9,0,1,0,1,13,0.0,0.0,3.59589,1,1,0,0.0,0.0,1.279792,484,13
2,1,30,12,1,0,0,0,13,0.0,0.0,24.90950,1,1,0,0.0,0.0,3.215249,900,13
3,1,27,11,1,0,0,1,13,0.0,0.0,7.50615,1,1,0,0.0,0.0,2.015723,729,13
4,1,33,8,1,0,0,1,13,0.0,0.0,0.28979,1,1,0,0.0,0.0,-1.238599,1089,13


In [2]:
df.re74.median()

0.0

In [3]:
# Simple Comparision of Means
import statsmodels.api as sm
import numpy as np
mod = sm.OLS(y, sm.add_constant(np.c_[d], prepend=False))
res = mod.fit()
print(res.params[0])
print(res.bse[0])

1.794343073101108
0.6328535506357185


In [4]:
# Pooled Regression Adjustment
import statsmodels.api as sm
mod = sm.OLS(y, sm.add_constant(np.c_[d, x], prepend=False))
res = mod.fit()
print(res.summary())
print(res.params[0])
print(res.bse[0])

                            OLS Regression Results                            
Dep. Variable:                   re78   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.037
Method:                 Least Squares   F-statistic:                     3.161
Date:                Wed, 14 Dec 2022   Prob (F-statistic):            0.00171
Time:                        23:43:50   Log-Likelihood:                -1460.2
No. Observations:                 445   AIC:                             2938.
Df Residuals:                     436   BIC:                             2975.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.6826      0.632      2.663      0.0

In [93]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=1)
est = LinearDML(discrete_treatment=False, cv = 5)
est.fit(y.ravel(), d.ravel(), X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,1.76,0.67,2.629,0.009,0.448,3.073


In [92]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state=1)
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
est = LinearDML(discrete_treatment=False, model_y = RandomForestRegressor(), model_t = RandomForestRegressor(), cv = 2)
est.fit(y.ravel(), d.ravel(), X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,1.722,0.666,2.584,0.01,0.416,3.028


In [83]:
import numpy as np
from doubleml.datasets import make_plr_CCDDHNR2018
from doubleml import DoubleMLData

np.random.seed(1234)
dml_data_bonus = DoubleMLData(df, y_col='re78',
                                  d_cols='train',
                                  x_cols=['age','educ', 'black', 'hisp', 'married', 're74', 're75'])
print(dml_data_bonus)
from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LassoCV
learner = RandomForestRegressor(n_estimators = 500, max_features = 'auto', max_depth= 6)
learner = Ridge()
learner = MLPRegressor((500,100))
learner = GradientBoostingRegressor(n_estimators=500, learning_rate = 0.01)
learner = RandomForestRegressor(n_estimators = 500, max_features = 'sqrt', max_depth= 6)
ml_l_bonus = clone(learner)
ml_m_bonus = clone(learner)
learner = LassoCV()
ml_l_sim = clone(learner)
ml_m_sim = clone(learner)
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    u_hat = y - g_hat
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, u_hat)
    return psi_a, psi_b

from doubleml import DoubleMLPLR
np.random.seed(42)
obj_dml_plr_bonus = DoubleMLPLR(dml_data_bonus, ml_l_bonus, ml_m_bonus, n_folds = 10)
obj_dml_plr_bonus.fit();
print(obj_dml_plr_bonus)

================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: re78
Treatment variable(s): ['train']
Covariates: ['age', 'educ', 'black', 'hisp', 'married', 're74', 're75']
Instrument variable(s): None
No. Observations: 445

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Columns: 19 entries, train to mostrn
dtypes: float64(6), int64(13)
memory usage: 66.2 KB

================== DoubleMLPLR Object ==================

------------------ Data summary      ------------------
Outcome variable: re78
Treatment variable(s): ['train']
Covariates: ['age', 'educ', 'black', 'hisp', 'married', 're74', 're75']
Instrument variable(s): None
No. Observations: 445

------------------ Score & algorithm ------------------
Score function: partialling out
DML algorithm: dml2

------------------ Machine learner   ------------------
Learner ml_l: RandomForestRe

In [73]:
# DML regression - still yeilds unbiased estimate of ATE 
from econml.dml import LinearDML
est = LinearDML(random_state = 4, cv=10)
est.fit(y, d, X=None,W=x)
est.summary()

Coefficient Results:  X is None, please call intercept_inference to learn the constant!


,point_estimate,stderr,zstat,pvalue,ci_lower,ci_upper
cate_intercept,1.675,0.67,2.501,0.012,0.362,2.988
